In [1]:
import os

os.environ["PYSYN_CDBS"] = "/home/ruizhew/POSEIDON/inputs/stellar_grids/"
os.environ["POSEIDON_input_data"] = "/home/ruizhew/POSEIDON/inputs/"

In [2]:
from __future__ import absolute_import, unicode_literals, print_function
from POSEIDON.high_res import (
    fast_filter,
    loglikelihood_sysrem,
    get_rot_kernel,
    fit_uncertainties,
    fit_uncertainties_and_remove_outliers,
)
import math, os
import numpy as np
import pickle
import pickle
from scipy import constants
from numba import jit
from astropy.io import fits
from scipy import interpolate
from POSEIDON.core import (
    create_star,
    create_planet,
    define_model,
    make_atmosphere,
    read_opacities,
    wl_grid_constant_R,
    wl_grid_line_by_line,
    compute_spectrum,
)
from POSEIDON.constants import R_Sun
from POSEIDON.visuals import plot_stellar_flux
from POSEIDON.constants import R_J, M_J
import numpy as np
from spectres import spectres
from tqdm import tqdm
from multiprocessing import Pool
import h5py
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d, median_filter

/Users/Victini/miniforge3/envs/poseidon_hr/lib/python3.9/site-packages/pysynphot/locations.py:345: UserWarning: Extinction files not found in /home/ruizhew/POSEIDON/inputs/stellar_grids/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))
/Users/Victini/miniforge3/envs/poseidon_hr/lib/python3.9/site-packages/pysynphot/refs.py:117: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for /home/ruizhew/POSEIDON/inputs/stellar_grids/mtab/*_tmg.fits
  warnings.warn('No graph or component tables found; '
/Users/Victini/miniforge3/envs/poseidon_hr/lib/python3.9/site-packages/pysynphot/refs.py:124: UserWarning: No thermal tables found, no thermal calculations can be performed. No files found for /home/ruizhew/POSEIDON/inputs/stellar_grids/mtab/*_tmt.fits
  warnings.warn('No thermal tables found, '


In [3]:
from scipy.optimize import minimize
from sklearn.decomposition import TruncatedSVD

In [4]:
data = h5py.File("./data/WASP-121.h5", "a")

In [5]:
# chips = ["blue", "redl", "redu"]

# for chip in chips:
#     flux = data[chip]["flux"][:]
#     wl_grid = data[chip]["wl_grid"][:]
#     N_order, N_phi, N_wl = flux.shape
#     if "flux_blaze_corrected" in data[chip]:
#         continue
#     else:
#         blaze = np.zeros(flux.shape)
#         for k in range(N_order):
#             order = flux[k]
#             median = np.median(order, axis=0)
#             order_norm = order / median
#             blaze[k] = order_norm

#         for det in range(N_order):
#             for phi in range(N_phi):
#                 blaze[det][phi] = median_filter(blaze[det][phi], size=501)

#         for det in range(N_order):
#             for phi in range(N_phi):
#                 blaze[det][phi] = gaussian_filter1d(blaze[det][phi], sigma=100)

#         flux = flux / blaze

#         print("Finish blaze correction")
#         data[chip].create_dataset("flux_blaze_corrected", data=flux)

#     if (
#         "Bs" in data[chip]
#         and "residuals" in data[chip]
#         and "uncertainties" in data[chip]
#     ):
#         continue
#     else:
#         uncertainties = fit_uncertainties(flux)
#         print("Finish fit uncertainties")

#         for i in range(N_order):
#             order = flux[i]
#             # out_of_transit_frames = np.concatenate((order[:20], order[-20:]), axis=0)
#             median = np.median(order, axis=0)
#             order_norm = order / median
#             uncertainty = uncertainties[i]

#             uncertainty_norm = uncertainty / median

#             uncertainties[i] = uncertainty_norm
#             flux[i] = order_norm

#         residuals, Us = fast_filter(flux, uncertainties, N_iter=15)

#         Bs = np.zeros((N_order, N_phi, N_phi))

#         for j in range(N_order):
#             U = Us[j]
#             L = np.diag(1 / np.mean(uncertainties[j], axis=-1))
#             B = U @ np.linalg.pinv(L @ U) @ L
#             Bs[j] = B

#         data[chip].require_dataset("residuals", shape=residuals.shape, dtype=float)
#         data[chip]["residuals"][:] = residuals
#         data[chip].require_dataset("Bs", shape=Bs.shape, dtype=float)
#         data[chip]["Bs"][:] = Bs
#         data[chip].require_dataset(
#             "uncertainties", shape=uncertainties.shape, dtype=float
#         )
#         data[chip]["uncertainties"][:] = uncertainties

In [5]:
chips = ["blue", "redl", "redu"]

for chip in chips:
    flux = data[chip]["flux"][:]
    wl_grid = data[chip]["wl_grid"][:]
    N_order, N_phi, N_wl = flux.shape
    if "flux_blaze_corrected" in data[chip]:
        flux = data[chip]["flux_blaze_corrected"][:]
    else:
        blaze = np.zeros(flux.shape)
        for k in range(N_order):
            order = flux[k]
            median = np.median(order, axis=0)
            order_norm = order / median
            blaze[k] = order_norm

        for det in range(N_order):
            for phi in range(N_phi):
                blaze[det][phi] = median_filter(blaze[det][phi], size=15)

        for det in range(N_order):
            for phi in range(N_phi):
                blaze[det][phi] = gaussian_filter1d(blaze[det][phi], sigma=50)

        flux = flux / blaze

        print("Finish blaze correction")
        data[chip].create_dataset("flux_blaze_corrected", data=flux)

    uncertainties = fit_uncertainties(flux)

    residuals, Us = fast_filter(flux, uncertainties, N_iter=10)

    model = flux - residuals

    residuals_2020 = flux / model - 1
    uncertainties = uncertainties / model

    if "uncertainties" in data[chip]:
        data[chip]["uncertainties"][:] = uncertainties
    else:
        data[chip].create_dataset("uncertainties", data=uncertainties)

    data[chip].require_dataset(
        "residuals_2020", shape=residuals_2020.shape, dtype=float
    )
    data[chip]["residuals_2020"][:] = residuals_2020

0.34674688269679577 233.05860530136158
0.31301649682054783 259.74873166965057
0.3347870272723711 259.3448484750303
0.3542486745660366 255.83499019842696
0.3363525375066892 262.09302843944545
0.36127372956838155 252.9687647883189
0.3538895907697627 257.1925104261539
0.3490046946331491 257.3246387316229
0.3429834689585245 260.8236011457467
0.3279206239447458 265.5847958638601
0.32447906813031996 264.1914699434165
0.33329363771820597 259.99798335008785
0.31813650091090706 266.9749165642313
0.30377843688499145 270.77128709142664
0.29612554360091914 273.52926340596144
0.3068598895379319 266.3212063846946
0.3064508886205523 265.7644861952833
0.27794783592090866 284.67988326243005
0.268293317260309 293.0811492625305
0.2685706553495375 288.93299219882545
0.2709465682703643 288.2778851424328
0.27329932082693376 286.2358993500822
0.27263271445349874 287.307813158777
0.2762483876037293 285.32757424449767
0.28355047369380737 275.94899598512234
0.2821260581158592 277.16768491053165
0.28514695856904

In [ ]:
transit_weight = pickle.load(open("./data/WASP-121b/transit_weight.pic", "rb"))

In [ ]:
data["redu"].create_dataset("transit_weight", data=transit_weight)

In [ ]:
data["redl"].create_dataset("transit_weight", data=transit_weight)

In [ ]:
data["blue"].create_dataset("transit_weight", data=transit_weight)

In [6]:
data.close()

In [ ]:
for key in data.keys():
    print(data[key])

In [ ]:
plt.imshow(data["redl"]["residuals_2020"][0], aspect="auto", cmap="viridis")

In [ ]:
import pickle

Bs, residuals = pickle.load(open("./data/WASP-121b/fast_filter.pic", "rb"))
uncertainties = pickle.load(open("./data/WASP-121b/uncertainties.pic", "rb"))

In [ ]:
plt.imshow(residuals[0], aspect="auto", cmap="viridis")

In [ ]:
plt.plot(data["blue"]["uncertainties"][10][0])
plt.plot(uncertainties[10][0])
plt.xlim(2000, 2400)

In [ ]:
plt.plot(data["redl"]["residuals_2020"][4][0])
plt.plot(data["blue"]["residuals"][20][0])
# plt.ylim(-0.1, 0.1)
plt.xlim(2000, 2300)